In [ ]:
import os
import pickle

os.environ["PRIVBAYES_BIN"] = "./ydnpd/harness/synthesis/privbayes/mac_bin"

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display, Markdown
import wandb
import tqdm

import ydnpd
from additional import ADDITIONAL_EXPERIMENTS, ADDITIONAL_PATH

In [2]:
ADDITIONAL_DATASETS = sum(list(ADDITIONAL_EXPERIMENTS.values()), [])

WANDB_GROUPS = ["acs-core", "edad-core", "we-core"]

## Tasks

In [ ]:
total_task_size = 0
for idx, task in enumerate(ydnpd.span_utility_tasks(additional_datasets=ADDITIONAL_DATASETS), start=1):
    print(f"{idx:2} {task}")
    total_task_size += task.size()
print(f"\nTotal task size: {total_task_size}")

## Load Results from W&B

In [ ]:
api = wandb.Api(timeout=30)

utility_tasks_results = []

for group in WANDB_GROUPS:

    runs = api.runs(path=f"shlomihod/ydnpd-harness", filters={"group": group})

    for run in tqdm.tqdm(runs):
        for step_data in run.history().to_dict(orient="records"):

            # TODO: why does this happen?
            if step_data["_step"] >= ydnpd.harness.config.NUM_RUNS:
                print(f"Run {run.id} has more than {ydnpd.config.NUM_RUNS} steps")
                continue

            for metric in step_data:
                if not metric.startswith("_") and step_data[metric] in ["NaN"]:
                    print(f"Run {run.id} with NaN at metric {metric}")
                    step_data[metric] = np.nan

            utility_tasks_results.append(dict(run.config) | {"evaluation": step_data})

## Missing Runs

In [ ]:
ydnpd.analyze_grid_search_completeness(utility_tasks_results, ADDITIONAL_DATASETS)

## Utility-Related Tasks

In [14]:
for experiments_name, experiments in ydnpd.ALL_EXPERIMENTS.items():

      if (additional_experiment_datasets := ADDITIONAL_EXPERIMENTS.get(experiments_name)):
            additional_dataset_names = [dataset_name for dataset_name, _ in additional_experiment_datasets]
            experiments = ydnpd.Experiments(experiments.test_name,
                                            experiments.dev_names + additional_dataset_names)            

      display(Markdown(f"## {experiments_name.upper()}"))

      datasets = {}
      for name in experiments.dev_names:
            try:
                  df = ydnpd.load_dataset(name)[0]
            except ValueError:
                  df = ydnpd.load_dataset(name, ADDITIONAL_PATH)[0]
            datasets[name] = df
      g = ydnpd.plot_distribution_distances(datasets)
      display(g.fig)

      ogs = (ydnpd.UtilityTask
            .plot_overall(utility_tasks_results,
                          experiments,
                          epsilon_reference=4)
                                          
      )

      for g in ogs:
            g.show()
            
      for metric in ydnpd.EVALUATION_METRICS:
          if metric not in utility_tasks_results[0]["evaluation"]:
              continue

          display(Markdown(f"### {metric}"))

          gs = ydnpd.UtilityTask.plot(utility_tasks_results, experiments, metric=metric)

          for g in gs:
              display(g.fig)
              plt.close(g.fig)

      for metric in ydnpd.EVALUATION_METRICS:
          if metric not in utility_tasks_results[0]["evaluation"]:
              continue

          display(Markdown(f"### {metric}"))
          with pd.option_context("display.max_rows", None):
              display(
                  ydnpd.UtilityTask.evaluate(utility_tasks_results, experiments, metric)
                  .multiply(100)
                  .round(1)
              )